# Aufgabe 1: Rucksackproblem

In [121]:
def solve_backpack(weight_list, value_list, max_i, max_weight):
    """
    optimize the value given a maximum weight by choosing several items from the lists
    inputs:
        weight_list - (list) - list of positive floats representing weights
        value_list - (list) - list of positive floats representing value
            both lists must have equal length
    returns:
        (float) - maximum value of the chosen items
        (list) of ints - indices of the chosen items
    """
    # print(f"({max_i+1}, {max_weight})", end=", ")
    if max_i < 0: #no items left to choose
        return 0, []
    # if max_weight < 0: # 
    #     return -1e10, []
    vi, wi = value_list[max_i], weight_list[max_i]
    # calculate sub-problems:
    # don't choose the current item
    value_2, items_2 = solve_backpack(weight_list, value_list, max_i-1, max_weight)
    if wi > max_weight: # prevents the case of max_weigth < 0
        return value_2, items_2
    # choose the current item:
    value_1, items_1 = solve_backpack(weight_list, value_list, max_i-1, max_weight-wi)
    value_1 += vi
    # if the value of choosing the current item is greater than the value for not choosing it, return the value and list for when the item is chosen.
    if value_1 > value_2:
        return value_1, items_1+[max_i]
    return value_2, items_2

In [122]:
# Vorlesungsbeispiel:
import random
pairs = [(1,15), (5,10), (3,9), (4,5)]
random.shuffle(pairs)
weigths, values = list(zip(*pairs))
print("weigths: ", weigths)
print("values:  ", values)
print(*solve_backpack(weigths, values, len(weigths)-1, 8))

weigths:  (3, 4, 5, 1)
values:   (9, 5, 10, 15)
(4, 8), (3, 8), (2, 8), (1, 8), (0, 8), (0, 5), (1, 4), (0, 4), (0, 1), (2, 3), (1, 3), (0, 3), (0, 0), (3, 7), (2, 7), (1, 7), (0, 7), (0, 4), (1, 3), (0, 3), (0, 0), (2, 2), (1, 2), (0, 2), 29 [0, 1, 3]


In [112]:
weigths = [5, 4, 6, 3]
values = [10, 40, 30, 50]
print(*solve_backpack(weigths, values, len(weigths)-1, 12))

100 [0, 1, 3]


# Aufgabe 2
## a)

Wir suchen die bedingte Übergangsmatrix $P$ für das Matchstick Game, wenn der Agent, wenn möglich, immer die Aktion selbst wählt und die Umgebung zufällig gleichverteilt 1,2 oder 3 als Aktion wählt.

Wie in der Vorlesung beginnt das Spiel mit 21 Stäbchen. 

Es werden in jeder Runde immer 3 (2+1),4 (2+2) oder 5 (2+3) Stäbchen weniger, jeweils mit Wahrscheinlichkeit $\frac{1}{3}$.

Es gibt wieder 22 Zustände: Im $n$-ten Zustand sind noch genau $n$ Stäbchen verfügbar ($n \in \{0, 1, ..., 21\}).

Dabei ist Zustand 20 und 19 aber nicht erreichbar, da vom Startzustand (21) aus durch die vorgegebene Policy ($A_t = 2$) mindestens 3 Stäbchen genommen werden.
Außerdem kann von keinem Zustand aus durch eine legale Aktion Zustand 21 erreicht werden.

Der Zustand $n$ wird dargestellt als ein 22-Tupel mit einer $1$ an Stelle $n$ und sonst nur Nullen. So kann mit $s \cdot P$ die Wahrscheinlichkeit für jeden Zustandsübergang von $s$ nach $s'$ erhalten werden.

Die Matrix $P$ ist also wieder eine $22\times 22$ Matrix.

Betrachten wir den Zustand $s = n$, dann sind mit der gegebenen Policy $A_t = 2$ nur die Folgezustände $n-3$, $n-4$ und $n-5$ möglich, außer es sind nicht mehr genügend Stäbchen vorhanden. Die drei Möglicheiten sind dabei alle gleich wahrscheinlich.

Also ergibt sich folgende Matrix:
$$
P = \left(
\begin{array}{cccccccccccccccccccccc}
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 0
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 1
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 2
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 3
 \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 4
 \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 5
 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 6
 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 7
 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 8
 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 9
 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 10
 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 11
 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 12
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 13
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 14
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 15
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 16
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 17
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 18
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 \\ % 19
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 \\ % 20
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 \\ % 21
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 \\ % 22
\end{array}
\right)
$$

Besonders sind hier die Fälle $0$ bis $4$, die abweichende Wahrscheinlichkeiten beinhalten. Dazu hier die Erklärung:
- In Zustand 4 nimmt der Agent zuerst 2 Stäbchen. Es bleiben also noch 2. Spieler 2 nimmt dann mit gleicher Wahrscheinlichkeit 1 oder 2 Stäbchen, da drei nicht mehr möglich sind. Somit ergibt sich die Wahrscheinlichkeit von jeweils $\frac{1}{2}$ für die Übergänge in Zustand $0$ bzw. $1$.

- In Zustand 3 nimmt der Agent 2 Stäbchen, sodass Spieler 2 nur die Option bleibt das letzte zu nehmen, sodass ein Übergang zu Zustand $0$ garantiert ist.
- In Zustand 2 nimmt der Agent 2 Stäbchen und verliert damit sofort (erreicht Zustand $0$).
- In Zustand 1 kann der Agent keine 2 Stäbchen nehmen, nimmt also nur das eine verbleibende und verliert damit sofort (erreicht Zustand $0$).

Außerdem ist in der Matrix ersichtlich, dass die Zustände $21$, $20$ und $19$ nicht Folgezustände anderer sind. Die vorletzten zwei Zeilen (19 und 20) sind also für die gegebenen Strategien eigentlich nie relevant.

## b)

Wir wollen prüfen ob der Bellman-Erwartungs-Operator $T^\pi$ die Voraussetzungen für den Banachschen Fixpunktsatz erfüllt, also eine Kontraktion ist.

Dazu betrachten wir den metrischen Raum $\mathbb{R}^{22}$ und definieren $T^\pi(v) := R^\pi + \gamma P^\pi v$.
Wir gehen davon aus, dass alle $v$ in einer kompakten Teilmenge von $\mathbb{R}^{22}$ liegen, ein Zustand also nur begrenzt große Wertung hat.

Nun wollen prüfen, ob $T^\pi$ eine $\gamma$-Kontraktion ist.

### Beweis:

- $R^\pi \in \mathbb{R}^{22}$ ist ein konstanter Vektor.
- $P^\pi$ ist wie in a) definiert die Zustands-Übergangsmatrix.
- $\gamma \in [0,1]$

Seien $a,b \in \mathbb{R}^{22}$ mit $d(a,b) = \sqrt{(b_1 - a_1)^2 + ... (b_{22}-a_{22})^2}$.

Zu zeigen ist nun also dass $d(T^\pi(a), T^\pi(b)) \leq \gamma \, d(a,b)$ gilt.

Einsetzen Definition von $T^\pi$ ergibt:
$$
\begin{align}
d(T^\pi(a), T^\pi(b)) &= \big| T^\pi(b)-T^\pi(a) \big|                                          \\
                      &= \big| ( R^\pi + \gamma P^\pi b )  -  ( R^\pi + \gamma P^\pi a ) \big|  \\
                      &= \big|  R^\pi - R^\pi  +  \gamma P^\pi b - \gamma P^\pi a  \big|        \\
                      &= \big|  \gamma (P^\pi b - P^\pi a)  \big|                               \\
                      &\overset{\gamma \, \geq \, 0}{=} \gamma \, \big| P^\pi b - P^\pi a \big| \\
                      &= \gamma \, \big| P^\pi (b - a) \big|                                    \\
                      &\not\leq \gamma \, |b-a|                                                 \\
                      &= \gamma \, d(a,b)
\end{align}
$$

Diese Ungleichung gilt nicht. Betrachte dazu folgendes Gegenbeispiel:


Sei $a = 0 \in \mathbb{R}^{22}$ der Nullvektor und $e_1 = (1,0,0,...,0) \in \mathbb{R}^{22}$ der Einheitsvektor.

Beide Vektoren stellen denkbare Bewertungsfunktionen $V$ dar.

Dann gilt aber $P^\pi\,(e_1-a) = P^\pi e_1 = (1,1,1,1,\frac{1}{2}, \frac{1}{3}, 0,0, ...,0)$. Damit gilt offensichtlich $|P^\pi e_1| > |e_1|$, sodass die Ungleichung widerlegt ist.

Daher ist die Funktion $T^\pi$ **keine** Kontraktion.